

Chapter 1: Exploring the GDALDataset class
====================================

## What is GDAL?


Some class methods include:

- `GetDriver`
- `GetRasterBand`
- `GetGeoTransform`
- `GetProjection`
- `GetSubDatasets`

    
Another suite of class methods allow you to set class attributes. These include:

- `SetGeoTransform`
- `SetProjection`

which allow you to modify the geographic projection and location of the image.

### Import module

In [1]:
# Import the Python 3 print function
from __future__ import print_function

# Import the "gdal" submodule from within the "osgeo" module
from osgeo import gdal

# We can check which version we're running by printing the "__version__" variable
print("GDAL's version is: " + gdal.__version__)
print(gdal)

GDAL's version is: 2.3.3
<module 'osgeo.gdal' from 'C:\\Users\\Nilay Nishant\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\osgeo\\gdal.py'>



### Examples
#### Open an image

In [3]:
# Open a GDAL dataset
dataset = gdal.Open('../../example/LE70220491999322EDC01_stack.gtif', gdal.GA_ReadOnly)

print(dataset)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000027BC6B2BF90> >



#### Get Image attributes

In [4]:
# How many bands does this image have?
num_bands = dataset.RasterCount
print('Number of bands in image: {n}\n'.format(n=num_bands))

# How many rows and columns?
rows = dataset.RasterYSize
cols = dataset.RasterXSize
print('Image size is: {r} rows x {c} columns\n'.format(r=rows, c=cols))

# Does the raster have a description or metadata?
desc = dataset.GetDescription()
metadata = dataset.GetMetadata()

print('Raster description: {desc}'.format(desc=desc))
print('Raster metadata:')
print(metadata)
print('\n')

# What driver was used to open the raster?
driver = dataset.GetDriver()
print('Raster driver: {d}\n'.format(d=driver.ShortName))

# What is the raster's projection?
proj = dataset.GetProjection()
print('Image projection:')
print(proj + '\n')

# What is the raster's "geo-transform"
gt = dataset.GetGeoTransform()
print('Image geo-transform: {gt}\n'.format(gt=gt))

Number of bands in image: 8

Image size is: 250 rows x 250 columns

Raster description: ../../example/LE70220491999322EDC01_stack.gtif
Raster metadata:
{'AREA_OR_POINT': 'Area', 'Band_1': 'band 1 reflectance', 'Band_2': 'band 2 reflectance', 'Band_3': 'band 3 reflectance', 'Band_4': 'band 4 reflectance', 'Band_5': 'band 5 reflectance', 'Band_6': 'band 7 reflectance', 'Band_7': 'band 6 temperature', 'Band_8': 'Band 8'}


Raster driver: GTiff

Image projection:
PROJCS["WGS 84 / UTM zone 15N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-93],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Ea

#### Read Image raster bands


In [6]:
# Open the blue band in our image
blue = dataset.GetRasterBand(1)

print(blue)

<osgeo.gdal.Band; proxy of <Swig Object of type 'GDALRasterBandShadow *' at 0x0000027BCDF569C0> >


### Raster Stats

In [16]:
band_max, band_min, band_mean, band_stddev = blue.GetStatistics(0, 1)
print('Band range: {minimum} - {maximum}'.format(maximum=band_max,
                                                 minimum=band_min))
print('Band mean, stddev: {m}, {s}\n'.format(m=band_mean, s=band_stddev))


Band range: 1810.0 - 198.0
Band mean, stddev: 439.015984, 139.7168287663



#### From GDAL to Numpy

In [17]:
# No alias
import numpy
print(numpy.__version__)

# Alias or rename to "np" -- a very common practice
import numpy as np
print(np.__version__)

1.14.5
1.14.5


In order to read our band into one of these wonderful `np.array` objects, we will use the `ReadAsArray` method from our `GDALRasterBand` object:

In [11]:
help(blue.ReadAsArray)

Help on method ReadAsArray in module osgeo.gdal:

ReadAsArray(xoff=0, yoff=0, win_xsize=None, win_ysize=None, buf_xsize=None, buf_ysize=None, buf_type=None, buf_obj=None, resample_alg=0, callback=None, callback_data=None) method of osgeo.gdal.Band instance
    Reading a chunk of a GDAL band into a numpy array. The optional (buf_xsize,buf_ysize,buf_type)
    parameters should generally not be specified if buf_obj is specified. The array is returned



### Read in the entire image:

In [12]:
blue_data = blue.ReadAsArray()

print(blue_data)
print()
print('Blue band mean is: {m}'.format(m=blue_data.mean()))
print('Size is: {sz}'.format(sz=blue_data.shape))

[[569 526 569 ... 311 289 311]
 [568 589 568 ... 267 332 332]
 [546 525 589 ... 311 311 311]
 ...
 [499 543 478 ... 306 349 372]
 [520 520 543 ... 328 372 393]
 [543 564 543 ... 393 414 436]]

Blue band mean is: 439.015984
Size is: (250, 250)


Let's say we want to read all of our bands into one 3 dimensional (nrow x ncol x nband) dataset. We will begin by initializing a 3 dimensional array. Next, we will loop over all bands in our raster image dataset and read them into our newly allocated 3 dimensional array:

In [13]:
# Initialize a 3d array -- use the size properties of our image for portability!
image = np.zeros((dataset.RasterYSize, dataset.RasterXSize, dataset.RasterCount))

# Loop over all bands in dataset
for b in range(dataset.RasterCount):
    # Remember, GDAL index is on 1, but Python is on 0 -- so we add 1 for our GDAL calls
    band = dataset.GetRasterBand(b + 1)
    
    # Read in the band's data into the third dimension of our array
    image[:, :, b] = band.ReadAsArray()

print(image)
print(image.dtype)

[[[ 569.  886.  758. ...  988. 2686.    0.]
  [ 526.  886.  758. ...  988. 2686.    0.]
  [ 569.  798.  776. ...  961. 2686.    0.]
  ...
  [ 311.  452.  356. ...  780. 2487.    0.]
  [ 289.  452.  297. ...  647. 2487.    0.]
  [ 311.  430.  337. ...  674. 2436.    0.]]

 [[ 568.  886.  815. ... 1014. 2686.    0.]
  [ 589.  929.  853. ... 1014. 2686.    0.]
  [ 568.  907.  910. ... 1014. 2686.    0.]
  ...
  [ 267.  430.  318. ...  674. 2487.    0.]
  [ 332.  452.  337. ...  594. 2487.    0.]
  [ 332.  452.  395. ...  727. 2487.    0.]]

 [[ 546.  864.  834. ...  988. 2637.    0.]
  [ 525.  886.  834. ...  988. 2637.    0.]
  [ 589.  928.  891. ...  988. 2637.    0.]
  ...
  [ 311.  452.  356. ...  727. 2487.    0.]
  [ 311.  475.  395. ...  727. 2487.    0.]
  [ 311.  475.  356. ...  753. 2487.    0.]]

 ...

 [[ 499.  851.  766. ... 1124. 2785.    0.]
  [ 543.  829.  747. ... 1387. 2735.    0.]
  [ 478.  829.  766. ... 1414. 2735.    0.]
  ...
  [ 306.  473.  336. ...  753. 2537.    